[See docs/DISCLAIMER_SNIPPET.md](../../../docs/DISCLAIMER_SNIPPET.md)

# Alpha-AGI Business v2 • Colab Notebook

Run a compact **Alpha-Factory** orchestrator with two demonstration agents.
Works fully offline or upgrades automatically when `OPENAI_API_KEY` is set.

## 0 · Runtime check

In [ ]:
!nvidia-smi -L || echo 'GPU not detected - running on CPU'

## 1 · Clone repo & install dependencies

In [ ]:
%%bash
set -e
if [ ! -d AGI-Alpha-Agent-v0 ]; then
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git
fi
cd AGI-Alpha-Agent-v0
pip -q install openai_agents fastapi uvicorn gradio

## 2 · (Optional) Configure your OpenAI API key

In [ ]:
import os, getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key (leave blank for offline): ')

## 3 · Launch orchestrator

In [ ]:
import subprocess, re, queue, threading, sys, time
from IPython.display import display, IFrame

root = 'AGI-Alpha-Agent-v0/alpha_factory_v1/demos/alpha_agi_business_2_v1'
proc = subprocess.Popen([sys.executable, '-m', 'alpha_agi_business_2_v1'], cwd=root, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

q = queue.Queue()

def _tail():
    for line in proc.stdout:
        print(line, end='')
        m = re.search(r'http://localhost:(\d+)/docs', line)
        if m:
            q.put(int(m.group(1)))

threading.Thread(target=_tail, daemon=True).start()
port = q.get()
print(f'⏳ Waiting for REST docs on port {port}...')
for _ in range(40):
    try:
        import requests
        if requests.get(f'http://localhost:{port}/healthz').status_code == 200:
            break
    except Exception:
        time.sleep(0.2)

iframe = IFrame(src=f'http://localhost:{port}/docs', width='100%', height=600)
display(iframe)

## 4 · Explore the API

In [ ]:
import requests
base = f'http://localhost:{port}'
print('Available agents:', requests.get(f'{base}/agents').json())
print('Trigger planning:', requests.post(f'{base}/agent/planning/trigger').json())

## 5 · Graceful shutdown

In [ ]:
proc.terminate(); print('✅ Orchestrator stopped')